In [1]:
import requests
import pandas as pd
from functools import *
import numpy as np
import nltk
%load_ext autoreload
%autoreload 2
import recip as rr
import dill
import submatcher as sm
import hashlib

In [2]:
inp = ["flour","butter","milk", "egg", "cheese"]

In [3]:
allr = rr.GetRecipies(inp, recnum = 9)


Recipe cache found
recipe Fluffy Mozzarella Biscuits cache found
recipe Cheese Muffins cache found
recipe Cabrales Cheese Souffles with Endive and Asian Pear Salad cache found
recipe Berry sweet empanadas cache found
recipe Ocean-themed Vanilla Cupcakes for Broadway’s "Anything Goes cache found
recipe Cheesy Popovers cache found
recipe Fancy French Cheese Puff Fake-out cache found
recipe Parmesan Puffs cache found
recipe Cheese Puffs cache found
recipe Parmesan Herb Popovers Recipe cache found


In [9]:
tbdf = sm.makesubdict("https://www.allrecipes.com/article/common-ingredient-substitutions/")

In [12]:
tbdf['tou']

0     1/2 teaspoon cinnamon, 1/4 teaspoon ginger, an...
1           1 tablespoon flour OR 1 teaspoon cornstarch
2       1 cup pancake mix OR 1 cup Easy Biscuit Mixture
3     1/4 teaspoon baking soda plus 1/2 teaspoon cre...
4     4 teaspoons baking powder OR 1 teaspoon potass...
5        1 cup nonalcoholic beer OR 1 cup chicken broth
6     1 teaspoon imitation brandy extract plus enoug...
7     1 cup cracker crumbs OR 1 cup matzo meal OR 1 ...
8     1 bouillon cube plus 1 cup boiling water OR 1 ...
9     1 cup white sugar plus 1/4 cup molasses and de...
10    1 cup margarine OR 1 cup shortening plus 1/2 t...
11    1 cup shortening OR 7/8 cup vegetable oil OR 7...
12    1 cup yogurt OR 1 tablespoon lemon juice or vi...
13    1 cup shredded Colby cheddar OR 1 cup shredded...
14                   1 tablespoon chopped fresh parsley
15    1 cup canned or homemade chicken broth or stoc...
16    1 (1-ounce) square of unsweetened chocolate pl...
17    3 tablespoons unsweetened cocoa plus 1 tab

In [5]:
import json
with open('testout.json', 'w', encoding='utf-8') as f:
    json.dump(allr.to_json(), f)